# Sprint
## 深層学習スクラッチ 畳み込みニューラルネットワーク1

## 1.このSprintについて

### Sprintの目的
スクラッチを通してCNNの基礎を理解する

### どのように学ぶか
スクラッチで1次元用畳み込みニューラルネットワークを実装した後、学習と検証を行なっていきます。

## 2.1次元の畳み込みニューラルネットワークスクラッチ

**畳み込みニューラルネットワーク（CNN）**のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


このSprintでは1次元の**畳み込み層**を作成し、畳み込みの基礎を理解することを目指します。次のSprintでは2次元畳み込み層とプーリング層を作成することで、一般的に画像に対して利用されるCNNを完成させます。


クラスの名前はScratch1dCNNClassifierとしてください。クラスの構造などは前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。


### 1次元畳み込み層とは
CNNでは画像に対しての2次元畳み込み層が定番ですが、ここでは理解しやすくするためにまずは1次元畳み込み層を実装します。1次元畳み込みは実用上は自然言語や波形データなどの**系列データ**で使われることが多いです。


畳み込みは任意の次元に対して考えることができ、立体データに対しての3次元畳み込みまではフレームワークで一般的に用意されています。


### データセットの用意
検証には引き続きMNISTデータセットを使用します。1次元畳み込みでは全結合のニューラルネットワークと同様に平滑化されたものを入力します。

### 【問題1】
#### チャンネル数を1に限定した1次元畳み込み層クラスの作成
チャンネル数を1に限定した1次元畳み込み層のクラスSimpleConv1dを作成してください。基本構造は前のSprintで作成した全結合層のFCクラスと同じになります。なお、重みの初期化に関するクラスは必要に応じて作り変えてください。Xavierの初期値などを使う点は全結合層と同様です。


ここでは**パディング**は考えず、**ストライド**も1に固定します。また、複数のデータを同時に処理することも考えなくて良く、バッチサイズは1のみに対応してください。この部分の拡張はアドバンス課題とします。


フォワードプロパゲーションの数式は以下のようになります。

$$
a_i = \sum_{s=0}^{F-1}x_{(i+s)}w_s+b
$$

$a_i$ : 出力される配列のi番目の値


$F$ : フィルタのサイズ


$x_{(i+s)}$ : 入力の配列の(i+s)番目の値


$w_s$ : 重みの配列のs番目の値


$b$ : バイアス項


全てスカラーです。


次に更新式です。ここがAdaGradなどに置き換えられる点は全結合層と同様です。

$$
w_s^{\prime} = w_s - \alpha \frac{\partial L}{\partial w_s} \\
b^{\prime} = b - \alpha \frac{\partial L}{\partial b}
$$

$\alpha$ : 学習率


$\frac{\partial L}{\partial w_s}$ : $w_s$に関する損失$L$の勾配


$\frac{\partial L}{\partial b}$ : $b$に関する損失$L$の勾配


勾配$\frac{\partial L}{\partial w_s}$や$\frac{\partial L}{\partial b}$を求めるためのバックプロパゲーションの数式が以下です。

$$
\frac{\partial L}{\partial w_s} = \sum_{i=0}^{N_{out}-1} \frac{\partial L}{\partial a_i}x_{(i+s)}\\
\frac{\partial L}{\partial b} = \sum_{i=0}^{N_{out}-1} \frac{\partial L}{\partial a_i}
$$

$\frac{\partial L}{\partial a_i}$ : 勾配の配列のi番目の値


$N_{out}$ : 出力のサイズ


前の層に流す誤差の数式は以下です。

$$
\frac{\partial L}{\partial x_j} = \sum_{s=0}^{F-1} \frac{\partial L}{\partial a_{(j-s)}}w_s
$$

$\frac{\partial L}{\partial x_j}$ : 前の層に流す誤差の配列のj番目の値


ただし、$j−s<0$または$j−s>N_{out}−1$のとき$\frac{\partial L}{\partial a_{(j-s)}}=0$です。


全結合層との大きな違いは、重みが複数の特徴量に対して共有されていることです。この場合は共有されている分の誤差を全て足すことで勾配を求めます。計算グラフ上での分岐はバックプロパゲーションの際に誤差の足し算をすれば良いことになります。

In [1]:
import numpy as np

In [2]:
class SimpleConv1d:
    """
    チャンネル数を1に限定した1次元畳み込み層
    (パディングなし　ストライド1)
    """
    def __init__(self, filter_size, initializer, optimizer):
        self.optimizer = optimizer
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.W = initializer.W(filter_size)
        self.B = initializer.B()
        pass

    def forward(self, X):
        """
        フォワード
        """
        self.X_ = X

        A = np.zeros((X.shape[0]-self.W.shape[0]+1,))
        for i in range(A.shape[0]):
            A[i] = np.sum(X[i:i+self.W.shape[0]]*self.W) + self.B

        return A

    def backward(self, dA):
        """
        バックワード
        """
        # 更新
        dX = self.optimizer.update(self, dA)

        return dX

In [3]:
class SGDConv:
    """
    勾配降下法（畳み込み）
    """
    def __init__(self, lr):
        self.lr = lr

    def update(self, layer, dA):
        """
        ある層の重みやバイアスの更新
        """
        dB = np.sum(dA)
        dW = np.zeros(layer.W.shape[0])
        for s in range(dW.shape[0]):
            dW[s] = np.dot(layer.X_[s:s+dA.shape[0]], dA)
        
        dX = np.zeros(layer.input_size)
        dA_padding = np.pad(dA, (layer.W.shape[0]-1, max((0, layer.input_size-dA.shape[0]))), mode='constant')
        for s in range(dX.shape[0]):
            dX[s] = np.sum(dA_padding[s:s+layer.W.shape[0]]*layer.W[::-1])

        layer.B = layer.B - self.lr*dB
        layer.W = layer.W - self.lr*dW

        return dX

### 【問題2】
#### 1次元畳み込み後の出力サイズの計算
畳み込みを行うと特徴量の数が変化します。どのように変化するかは以下の数式から求められます。パディングやストライドも含めています。この計算を行う関数を作成してください。

$$
N_{out} =  \frac{N_{in}+2P-F}{S} + 1\\
$$

$N_{out}$ : 出力のサイズ（特徴量の数）


$N_{in}$ : 入力のサイズ（特徴量の数）


$P$ : ある方向へのパディングの数


$F$ : フィルタのサイズ


$S$ : ストライドのサイズ

In [67]:
def calc_output_size(input_size, padding_size, filter_size, stride_size):
    n_out = ((input_size+2*padding_size-filter_size) / stride_size) + 1
    return n_out

### 【問題3】
#### 小さな配列での1次元畳み込み層の実験
次に示す小さな配列でフォワードプロパゲーションとバックプロパゲーションが正しく行えているか確認してください。


入力x、重みw、バイアスbを次のようにします。

```python
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])
```

フォワードプロパゲーションをすると出力は次のようになります。

```python
a = np.array([35, 50])
```

次にバックプロパゲーションを考えます。誤差は次のようであったとします。

```python
delta_a = np.array([10, 20])
```

バックプロパゲーションをすると次のような値になります。

``` python
delta_b = np.array([30])
delta_w = np.array([50, 80, 110])
delta_x = np.array([30, 110, 170, 140])
```

In [5]:
# forward
x = np.array([1, 2, 3, 4])
w = np.array([3, 5, 7])
b = np.array([1])

a = np.zeros((x.shape[0]-w.shape[0]+1,))
for i in range(a.shape[0]):
    a[i] = np.sum(x[i:i+w.shape[0]]*w) + b

print(a)
## a = np.array([35, 50])

[35. 50.]


In [6]:
# backward
delta_a = np.array([10, 20])

delta_b = np.sum(delta_a)
delta_w = np.zeros(w.shape[0])
for s in range(delta_w.shape[0]):
    delta_w[s] = np.dot(x[s:s+delta_a.shape[0]], delta_a)

delta_x = np.zeros(x.shape[0])
delta_a_padding = np.pad(delta_a, (w.shape[0]-1, max((0, x.shape[0]-delta_a.shape[0]))), mode='constant')
for s in range(delta_x.shape[0]):
    delta_x[s] = np.sum(delta_a_padding[s:s+w.shape[0]]*w[::-1])

print(delta_b)
## delta_b = np.array([30])
print(delta_w)
## delta_w = np.array([50, 80, 110])
print(delta_x)
## delta_x = np.array([30, 110, 170, 140])

30
[ 50.  80. 110.]
[ 30. 110. 170. 140.]


### 実装上の工夫
畳み込みを実装する場合は、まずはfor文を重ねていく形で構いません。しかし、できるだけ計算は効率化させたいため、以下の式を一度に計算する方法を考えることにします。

$$
a_i = \sum_{s=0}^{F-1}x_{(i+s)}w_s+b
$$

バイアス項は単純な足し算のため、重みの部分を見ます。

$$
\sum_{s=0}^{F-1}x_{(i+s)}w_s
$$

これは、xの一部を取り出した配列とwの配列の内積です。具体的な状況を考えると、以下のようなコードで計算できます。この例では流れを分かりやすくするために、各要素同士でアダマール積を計算してから合計を計算しています。これは結果的に内積と同様です。

```python
x = np.array([1, 2, 3, 4])
w = np.array([3, 5, 7])
a = np.empty((2, 3))
indexes0 = np.array([0, 1, 2]).astype(np.int)
indexes1 = np.array([1, 2, 3]).astype(np.int)
a[0] = x[indexes0]*w # x[indexes0]は([1, 2, 3])である
a[1] = x[indexes1]*w # x[indexes1]は([2, 3, 4])である
a = a.sum(axis=1)
````

ndarrayは配列を使ったインデックス指定ができることを利用した方法です。


また、二次元配列を使えば一次元配列から二次元配列が取り出せます。

```python
x = np.array([1, 2, 3, 4])
indexes = np.array([[0, 1, 2], [1, 2, 3]]).astype(np.int)
print(x[indexes]) # ([[1, 2, 3], [2, 3, 4]])
```

このこととブロードキャストなどをうまく組み合わせることで、一度にまとめて計算することも可能です。


畳み込みの計算方法に正解はないので、自分なりに効率化していってください。


**《参考》**


以下のページのInteger array indexingの部分がこの方法についての記述です。


[Indexing — NumPy v1.17 Manual](https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html)

### 【問題4】
#### チャンネル数を限定しない1次元畳み込み層クラスの作成
チャンネル数を1に限定しない1次元畳み込み層のクラスConv1dを作成してください。


例えば以下のようなx, w, bがあった場合は、

```python
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3]) # （出力チャンネル数）
```

出力は次のようになります。

```python
a = np.array([[16, 22], [17, 23], [18, 24]]) # shape(3, 2)で、（出力チャンネル数、入力チャンネル数）である。
```

入力が2チャンネル、出力が3チャンネルの例です。計算グラフを書いた上で、バックプロパゲーションも手計算で考えてみましょう。計算グラフの中には和と積しか登場しないので、微分を新たに考える必要はありません。


**《補足》**


チャンネル数を加える場合、配列をどういう順番にするかという問題があります。**(バッチサイズ、チャンネル数、特徴量数)**または**(バッチサイズ、特徴量数、チャンネル数)**が一般的で、ライブラリによって順番は異なっています。（切り替えて使用できるものもあります）


今回のスクラッチでは自身の実装上どちらが効率的かを考えて選んでください。上記の例ではバッチサイズは考えておらず、**(チャンネル数、特徴量数)**です。

In [72]:
class Conv1d:
    """
    チャンネル数を限定しない1次元畳み込み層
    """
    def __init__(self, initializer, optimizer, filter_size, padding_size=0, stride=1):
        self.optimizer = optimizer        
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.W = initializer.W(filter_size, output_size)
        self.B = initializer.B()
        self.filter_size = filter_size
        # kwargs
        self.padding_size = padding_size
        self.stride = stride

    def forward(self, X):
        """
        フォワード
        """
        # padding
        # axisは(バッチサイズ, チャンネル数, 特徴量)とする
        # 1チャンネルでも(m, 1, n)になるように調整する必要あり
        if self.padding_size > 0:
            # shapeが(m, n)で入ってきた場合の処理
            if len(X.shape) == 2: X = X[:, np.newaxis, :]
            X = np.pad(X, ((0, 0), (0, 0), (self.padding_size, self.padding_size)), mode='constant')
        
        # 逆伝搬用
        self.X_ = X
        # 特徴量のサイズ
        self.input_size = X.shape[-1]
        # 出力のサイズ
        self.output_size = calc_output_size(self.input_size,
                                            self.padding_size,
                                            self.filter_size,
                                            self.stride_size)

        # self.Wに対応するXのインデックス
        self.windows = []
        for i in range(0, X.shape[-1]-self.W.shape[-1]+1, self.stride):
            self.windows.append(list(range(i, i+self.W.shape[-1])))
        self.windows = np.array(self.windows)

        A = np.zeros((X.shape[0], self.W.shape[0], self.output_size))
        # batch内でループを行う
        for sample_num, sample in enumerate(X):
            ## X_tmp: バッチ内の1データに対応する
            X_tmp = sample
            # output channel
            for output_channel in range(self.W.shape[0]):
                # input channel
                for input_channel in range(X_tmp.shape[0]):
                    A[sample_num][output_channel] += np.sum(X_tmp[input_channel][windows]*self.W[output_channel,
                                                            input_channel],
                                                            axis=1)

        A += self.B[np.newaxis, :, np.newaxis]

        return A

    def backward(self, dA):
        """
        バックワード
        """
        # 更新
        dX = self.optimizer.update(self, dA)

        return dX

In [73]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3]) # （出力チャンネル数）
print(x)
print(w)
print(b, end='\n\n')

# self.Wに対応するXのインデックス
windows = np.array([list(range(i, i+w.shape[2])) for i in range(x.shape[1]-w.shape[-1]+1)])
print(windows, end='\n\n')

#for i,xch in enumerate(x):
#   print(w[:, i, :], end='\n\n')
#   print(xch[windows])
#   print(np.array([np.sum(w[:, i, :][j]*xch[windows]) for j in range(w.shape[0])])[:, np.newaxis])

# 入力チャンネル単位でself.Wを適用してから結合
#a = np.concatenate([np.array([np.sum(X_ch[windows]*w[:, i, :][j]) for j in range(w.shape[0])])[:, np.newaxis] for i, X_ch in enumerate(x)], axis=1) + b[:, np.newaxis]

output_size = 2
tmp_A = np.zeros((w.shape[0], output_size))
# output channel
for output_channel in range(w.shape[0]):
    # input channel
    for input_channel in range(x.shape[0]):
            tmp_A[output_channel] += np.sum(x[input_channel][windows]*w[output_channel, input_channel], axis=1)
            pass
print(tmp_A)


print(x[0][windows])
print(w[0, 0])
#print(np.sum(x[0][windows]*w[0, 0], axis=1))

#print(np.dot(x[0][windows], w[0]))
a = tmp_A + b[:, np.newaxis]
print(a)
print(a.shape)
# a = np.array([[16, 22], [17, 23], [18, 24]]) # shape(3, 2)で、（出力チャンネル数、特徴量数）である

[[1 2 3 4]
 [2 3 4 5]]
[[[1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]]]
[1 2 3]

[[0 1 2]
 [1 2 3]]

[[15. 21.]
 [15. 21.]
 [15. 21.]]
[[1 2 3]
 [2 3 4]]
[1. 1. 1.]
[[16. 22.]
 [17. 23.]
 [18. 24.]]
(3, 2)


**以下問題5, 6, 7についても`Conv1d`クラスを編集する。**

### 【問題5】
#### （アドバンス課題）パディングの実装
畳み込み層にパディングの機能を加えてください。1次元配列の場合、前後にn個特徴量を増やせるようにしてください。


最も単純なパディングは全て0で埋める ゼロパディング であり、CNNでは一般的です。他に端の値を繰り返す方法などもあります。


フレームワークによっては、元の入力のサイズを保つようにという指定をすることができます。この機能も持たせておくと便利です。なお、NumPyにはパディングの関数が存在します。


[numpy.pad — NumPy v1.17 Manual](https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html)

```python
# padding
if self.padding_size > 0:
    X = np.pad(X, [self.padding_size, self.padding_size], mode='constant')    
```

### 【問題6】
#### （アドバンス課題）ミニバッチへの対応
ここまでの課題はバッチサイズ1で良いとしてきました。しかし、実際は全結合層同様にミニバッチ学習が行われます。Conv1dクラスを複数のデータが同時に計算できるように変更してください。

In [27]:
y =  np.array([[[1,2,3], [2,3,4]], [[3,4,5], [4,5,6]], [[5,6,7], [6,7,8]]])
print(y)
print(np.pad(y, ((0, 0), (0, 0), (1, 1)), mode='constant'))
print(x[:, np.newaxis, :])
print(y[:, np.newaxis, :])

[[[1 2 3]
  [2 3 4]]

 [[3 4 5]
  [4 5 6]]

 [[5 6 7]
  [6 7 8]]]
[[[0 1 2 3 0]
  [0 2 3 4 0]]

 [[0 3 4 5 0]
  [0 4 5 6 0]]

 [[0 5 6 7 0]
  [0 6 7 8 0]]]
[[[1 2 3 4]]

 [[2 3 4 5]]]
[[[[1 2 3]
   [2 3 4]]]


 [[[3 4 5]
   [4 5 6]]]


 [[[5 6 7]
   [6 7 8]]]]


### 【問題7】
#### （アドバンス課題）任意のストライド数
ストライドは1限定の実装をしてきましたが、任意のストライド数に対応できるようにしてください。

```python
# self.Wに対応するXのインデックス
self.windows = []
for i in range(0, X.shape[1]-self.W.shape[2]+1, self.stride):
    self.windows.append(list(range(i, i+self.W.shape[2])))
self.windows = np.array(self.windows)
```

問5,6,7の拡張に対応したバックプロパゲーションを実装する。

```python
class SGDConv:
    """
    勾配降下法（畳み込み）
    """
    def __init__(self, lr):
        self.lr = lr

    def update(self, layer, dA):
        """
        ある層の重みやバイアスの更新
        """
        dB = np.sum(dA)
        dW = np.zeros(layer.W.shape[0])
        for s in range(dW.shape[0]):
            dW[s] = np.dot(layer.X_[s:s+dA.shape[0]], dA)
        
        dX = np.zeros(layer.input_size)
        dA_padding = np.pad(dA, (layer.W.shape[0]-1, max((0, layer.input_size-dA.shape[0]))), mode='constant')
        for s in range(dX.shape[0]):
            dX[s] = np.sum(dA_padding[s:s+layer.W.shape[0]]*layer.W[::-1])

        layer.B = layer.B - self.lr*dB
        layer.W = layer.W - self.lr*dW

        return dX
```

In [ ]:
class SGDConvMulti:
    """
    勾配降下法（畳み込み）

    """
    def __init__(self, lr):
        self.lr = lr

    def update(self, layer, dA):
        """
        ある層の重みやバイアスの更新
        dA.shape -> (出力チャンネル数, output_size)
        """
        output_size = calc_output_size(layer.input_size,
                                        layer.padding_size,
                                        layer.filter_size,
                                        layer.stride)
        # dB
        ## batch_sizeの軸に対して合計する
        ## -> output_sizeの軸に対して合計する
        dB = np.sum(np.sum(dA, axis=0), axis=-1)
        
        # dW (出力チャンネル数、入力チャンネル数、フィルタサイズ)にしたい

        dW = []
        for output_channel in range(layer.W.shape[0]):
            dW.append([])
            for input_channel in range(layer.W.shape[1])
                dW[output_channel].append([])
                dW[output_channel][input_channel]np.dot(layer.X_[s:s+dA.shape[0]], dA)
        
        dX = np.zeros(layer.input_size)
        dA_padding = np.pad(dA, (layer.W.shape[0]-1, max((0, layer.input_size-dA.shape[0]))), mode='constant')
        for s in range(dX.shape[0]):
            dX[s] = np.sum(dA_padding[s:s+layer.W.shape[0]]*layer.W[::-1])

        layer.B = layer.B - self.lr*dB
        layer.W = layer.W - self.lr*dW

        return dX